In [20]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go

def true_range(data):
    """
    Calcula o True Range dos dados.
    """
    high_minus_low = data['High'] - data['Low']
    high_minus_prev_close = np.abs(data['High'] - data['Close'].shift())
    low_minus_prev_close = np.abs(data['Low'] - data['Close'].shift())
    return np.max([high_minus_low, high_minus_prev_close, low_minus_prev_close], axis=0)

def media(data, periodo):
    """
    Calcula a média móvel simples usando a janela rolante do pandas.
    """
    return data['TrueRange'].rolling(periodo).mean()

def calculate_satrc_satrv(data, periodo, desvio):
    """
    Calcula os valores de SATRc e SATRv.
    """
    satrc = [0]
    satrv = [0]

    for i in range(1, len(data)):
        ATR = media(data, periodo)[i]

        SATRc1 = abs(data['Close'].iloc[i] - (ATR * desvio))
        SATRv1 = abs(data['Close'].iloc[i] + (ATR * desvio))

        if data['Close'].iloc[i] >= satrc[-1]:
            if SATRc1 > satrc[-1]:
                satrc.append(SATRc1)
            else:
                satrc.append(satrc[-1])
        else:
            satrc.append(SATRc1)

        if data['Close'].iloc[i] < satrv[-1]:
            if SATRv1 < satrv[-1]:
                satrv.append(SATRv1)
            else:
                satrv.append(satrv[-1])
        else:
            satrv.append(SATRv1)

    # Garantir que os comprimentos de satrc e satrv correspondam a data.index
    satrc = satrc[:-1]
    satrv = satrv[:-1]

    return satrc, satrv

# Configurar parametros (y - anos, mo - meses, wk - semanas, d - dias)
periodo_extr = '1y'
intervalo = '1d'

# Baixar os dados do índice Ibovespa
df = yf.download('^BVSP', period=periodo_extr, interval=intervalo)

# Exemplo de uso com dados do Ibovespa
data = df.copy()  # Copiar o DataFrame para evitar problemas de modificação direta

# Calcular o True Range e adicioná-lo ao DataFrame
data.loc[:, 'TrueRange'] = true_range(data)

periodo = 20
desvio = 2.50

satrc, satrv = calculate_satrc_satrv(data, periodo, desvio)

# Remover os primeiros registros que correspondem ao tamanho da variável período
data = data.iloc[periodo:].copy()
satrc = satrc[periodo:]
satrv = satrv[periodo:]

# Calcular a média móvel simples do preço de fechamento
data.loc[:, 'SMA'] = data['Close'].rolling(window=20).mean()

# Plotar os resultados
fig = go.Figure()

# Adicionar gráfico de candlesticks
fig.add_trace(go.Candlestick(x=data.index,
                             open=data['Open'],
                             high=data['High'],
                             low=data['Low'],
                             close=data['Close'],
                             name='Candlesticks',
                             increasing_line_width=2,
                             decreasing_line_width=2))

fig.add_trace(go.Scatter(x=data.index[1:], y=satrc, name='SATRc', line=dict(color='green')))
fig.add_trace(go.Scatter(x=data.index[1:], y=satrv, name='SATRv', line=dict(color='red')))

fig.add_trace(go.Scatter(x=data.index, y=data['SMA'], name='SMA', line=dict(color='black', dash='dash')))


fig.update_layout(title='Indicador Stop ATR',
                  xaxis_title='Data',
                  yaxis_title='Valores do Indicador')

fig.show()


[*********************100%***********************]  1 of 1 completed
